### Import Dependencies

In [14]:
import pandas as pd

import numpy as np
from scipy.spatial.distance import cdist, euclidean

### Import Data

In [16]:
file_to_load = "../json_files/LA_Certificated_Dispensary_Locations.csv"
disp_df = pd.read_csv(file_to_load)

In [20]:
disp_df["LOCATION"]

0      (33.7258, -118.2855)
1      (33.7744, -118.2629)
2      (33.7805, -118.2523)
3       (33.7816, -118.243)
4      (33.7871, -118.3081)
               ...         
415     (34.307, -118.4695)
416    (34.3075, -118.4614)
417    (34.3099, -118.4274)
418    (34.3106, -118.4755)
419    (34.3106, -118.4793)
Name: LOCATION, Length: 420, dtype: object

In [10]:
#todo
# Load crime data for each precinct into seperate numpy arrays

In [21]:
file_to_load = '../json_files/crime_2013.json'
crime_df = pd.read_json(file_to_load)

In [31]:
crime_df = crime_df.transpose()

In [41]:
crime_df

,Year,Date_Occur,LAPD_Area,LAPD_Area_Name,LAT,LON,Crime_Type,Premise,Weapon
0,2013,09/27/2013,8,West LA,34.062,-118.448,Burglary,Retail Location,None
1,2013,12/31/2013,1,Central,34.0363,-118.267,Theft,Other,None
2,2013,10/01/2013,14,Pacific,33.9557,-118.444,Burglary,House/Apartment/Yard,Personal Weapons
3,2013,08/22/2013,19,Mission,34.2282,-118.454,Robbery,Street/Alley/Sidewalk,Personal Weapons
4,2013,05/17/2013,21,Topanga,34.1865,-118.615,Assault,House/Apartment/Yard,Personal Weapons
...,...,...,...,...,...,...,...,...,...
126758,2013,10/18/2013,2,Rampart,34.055,-118.273,Assault,Parking Lot,Unk or Other
126759,2013,05/17/2013,10,West Valley,34.2022,-118.481,Burglary,House/Apartment/Yard,None
126760,2013,10/28/2013,12,77th Street,33.9856,-118.291,Assault,House/Apartment/Yard,Personal Weapons
126761,2013,07/15/2013,21,Topanga,34.1805,-118.597,Theft,Retail Location,None


### Calculate Geometric Median for precincts in LA

In [2]:
# https://stackoverflow.com/questions/30299267/geometric-median-of-multidimensional-points
# https://www.pnas.org/content/pnas/97/4/1423.full.pdf

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [12]:
# Test Case
a = np.array([[5,0],[-5,0],[0,5]])
geometric_median(a)

array([0.        , 2.88673462])